In [ ]:
# Install necessary libraries for the script
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install openai==0.28
!pip install GoogleSearch serpapi google-search-results
!pip install langchain_community
!pip install langchain.schema
!pip install langchain_openai
!pip install gradio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 458.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement GoogleSearch (from versions: none)
ERROR: No matching distribution 

In [ ]:
# Libraries Import
import os
import re
import json
import pickle
import requests
import numpy as np
import pandas as pd
from collections import Counter
from urllib.parse import urlparse

# Import necessary sklearn modules for machine learning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Import LangChain modules for building language model chains and agents
from langchain import OpenAI, LLMChain, PromptTemplate, SerpAPIWrapper
from langchain.agents import (AgentExecutor, AgentType, LLMSingleActionAgent, Tool, initialize_agent, load_tools)
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import BaseTool
from langchain.utilities import (GoogleSearchAPIWrapper, GoogleSerperAPIWrapper, WikipediaAPIWrapper)
from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOpenAI

# Import logging module for logging errors and other messages
import logging

In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
open_ai_key=os.environ["OPENAI_API_KEY"]
serper_api_key=os.environ["SERPER_API_KEY"]
llm_35=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)
llm_4 = ChatOpenAI(model_name="gpt-4o", temperature=0)

<ipython-input-3-372d154f0c4e>:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_35=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)


In [ ]:
# Define a dictionary of reliable sources with their respective reliability scores
RELIABLE_SOURCES = {
    "accion-ciudadana.org": 5.00,
    "tracoda.info": 5.00,
    "elconfidencial.com": 4.67,         # Spanish digital newspaper
    "elpais.com": 4.67,                 # Renowned Spanish newspaper
    "eldiario.es": 4.33,                # Independent Spanish media
    "abc.es": 4.00,                     # Spanish generalist newspaper
    "europapress.es": 4.00,             # Spanish news agency
    "lavanguardia.com": 4.33,           # Catalonia-based newspaper
    "20minutos.es": 4.00,               # Spanish daily news site
    "elespanol.com": 4.00,              # Spanish digital newspaper
    "huffingtonpost.es": 4.00,          # Spanish edition of the HuffPost
    "publico.es": 4.00,                 # Spanish progressive newspaper
    "larazon.es": 4.00,                 # Spanish daily newspaper
    "eldiarioar.com": 4.33,             # Argentine edition of eldiario.es
    "cnnespanol.cnn.com": 4.00,         # CNN's Spanish language site
    "bbc.com/mundo": 4.00,              # BBC Spanish edition
    "rtve.es": 4.33,                    # Spain's public broadcasting service
    "efe.com": 4.00,                    # Spanish news agency
    "clarin.com": 4.00,                 # Major Argentine newspaper
    "infobae.com": 4.00,                # Latin American news site
    "pagina12.com.ar": 4.00,            # Argentine newspaper
    "lavozdegalicia.es": 4.00,          # Regional newspaper from Galicia, Spain
    "laopinion.com": 4.00,              # US-based Spanish language newspaper
    "elespectador.com": 4.00,           # Colombian newspaper
    "semana.com": 4.00,                 # Colombian weekly magazine
    "elmundo.es": 4.00,                 # Major Spanish newspaper
    "nacion.com": 4.00,                 # Costa Rican newspaper
    "elpais.com.uy": 4.00,              # Uruguayan edition of El Pais
    "eltiempo.com": 4.00,               # Colombian newspaper
    "elpais.com.mx": 4.00,              # Mexican edition of El Pais
    "eluniversal.com.mx": 4.00,         # Major Mexican newspaper
    "laopiniondemurcia.es": 4.00,       # Regional newspaper from Murcia, Spain
    "es.wikipedia.org":5.00,
    "bloomberglinea.com":1.00,
    "latimes.com":1.00
}

In [ ]:
# Function to extract information using the Serper API
def extract_base_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    if domain.startswith("www."):
        domain = domain[4:]
    return domain

def info_extraction(subject, length=500, api_key=serper_api_key, min_search=30):
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": subject, "gl": "sv", "hl": "es-419", "num": min_search, "tbs": "qdr:m"})
    headers = {'X-API-KEY': api_key, 'Content-Type': 'application/json'}

    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        results = response.json()
        organic_results = results.get("organic", [])
    except requests.RequestException as e:
        print(f"Error during search: {e}")
        return [], [], []

    all_media_outlets, reliable_media, unreliable_media = [], [], []

    for result in organic_results:
        snippet = result.get('snippet')
        source_url = result.get('link')
        if snippet and source_url:
            base_domain = extract_base_domain(source_url)
            all_media_outlets.append(base_domain)

            if base_domain in RELIABLE_SOURCES:
                reliability_index = RELIABLE_SOURCES[base_domain]
                reliable_media.append({
                    "snippet": snippet[:length],
                    "source": base_domain,
                    "reliability_index": reliability_index
                })
            else:
                unreliable_media.append({"snippet": snippet[:length], "source": base_domain})
        else:
            print(f"Missing data in results: {result}")

    sorted_reliable = sorted(reliable_media, key=lambda x: x['reliability_index'], reverse=True)
    return all_media_outlets, sorted_reliable, unreliable_media

In [ ]:

# Base class for agents
class WrapperFrame_v1:
    def __init__(self, client):
        self.client = client

# Narrative Agents

# 1. Positive Sentiment Narratives
class ES_Agent_Positive(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.ES_agent = """
    Your task is to analyze and categorize a social media post {input_text} from El salvador according to predefined narrative themes that reflect positive sentiment with validated context.


    Narratives to Detect:

      ESP1: The Government’s Firm Approach Restoring Law and Order.
      Example: "The government's hardline security policies have restored law and order in El Salvador, reducing crime rates and bringing a sense of safety to local communities."
      Keywords: law and order, security, crime reduction, safety, firm approach.

      ESP2: El Salvador is Becoming a Model of Security and Strong Leadership in Fighting Gang Violence.
          Example: "El Salvador's leadership in combating gang violence is becoming a model for other countries in the region, showcasing the government's resolve and effectiveness."
          Keywords: security model, leadership, gang violence, regional example, safety.

      ESP3: Government Policies Attracting Support and International Investment.
          Example: "International companies are flocking to El Salvador, drawn by favorable government policies that support innovation and economic growth."
          Keywords: support, international investment, government policies, economic growth, innovation.

      ESP4: El Salvador at the Forefront of Economic Modernization and Cryptocurrency Adoption.
          Example: "El Salvador’s adoption of Bitcoin is positioning the country as a pioneer in financial innovation, paving the way for economic modernization."
          Keywords: cryptocurrency, Bitcoin, economic modernization, innovation, financial pioneer.

      ESP5: Bitcoin Policy Opening Doors for Innovation and Attracting Entrepreneurs and International Investors.
          Example: "El Salvador’s Bitcoin policy is attracting a new wave of entrepreneurs and international investors, driving technological innovation and boosting the economy."
          Keywords: Bitcoin policy, innovation, entrepreneurship, international investors, economic boost.

      ESP6: The Bitcoin Revolution Could Break the Country’s Dependence on Traditional Financial Systems.
          Example: "By embracing Bitcoin, El Salvador is reducing its reliance on traditional financial systems and promoting financial independence."
          Keywords: Bitcoin revolution, financial independence, traditional systems, innovation.

      ESP7: El Salvador Affirms Sovereignty and Emerges as a Regional Leader by Standing up to Foreign Pressure.
          Example: "El Salvador is asserting its sovereignty, showing resilience in the face of foreign pressure, and positioning itself as a leader in Central America."
          Keywords: sovereignty, leadership, resilience, foreign pressure, regional influence.

      ESP8: Independent Policies Lead to a New Era of Self-Sufficiency and Greater Regional Influence.
          Example: "The government's independent policies are fostering a new era of self-sufficiency, boosting El Salvador's regional influence."
          Keywords: independent policies, self-sufficiency, regional influence, leadership, progress.

      ESP9: Alternative Media Providing Truth Suppressed by Traditional Media.
          Example: "Alternative media outlets in El Salvador are giving voice to suppressed truths, countering narratives controlled by traditional media."
          Keywords: alternative media, suppressed truth, transparency, traditional media.

      ESP10: Government Promoting Transparency Through its Own Media Channels.
          Example: "The government’s media channels are promoting transparency by delivering unfiltered information directly to the public."
          Keywords: transparency, government media, unfiltered information, public access, direct communication.

      ESP11: Economic Growth Promised by Government Reforms and Innovation.
          Example: "Thanks to recent reforms and a focus on innovation, El Salvador is on the path to sustained economic growth."
          Keywords: economic growth, reforms, innovation, future prosperity, development.

      ESP12: Government Promoting Equality Policies and Empowering Women.
          Example: "New government policies aimed at promoting gender equality are empowering women across different sectors of society."
          Keywords: equality, women’s rights, empowerment, government policies, gender equality.

      ESP13: Support Programs for Vulnerable Women Reinforcing Commitment to Gender Equality.
          Example: "The government’s support programs for women in vulnerable situations are reinforcing its commitment to gender equality and social justice."
          Keywords: support programs, vulnerable women, gender equality, empowerment, social justice.

      ESP14: Policies for LGBT Inclusion and Respect for Rights.
          Example: "El Salvador is leading the way in LGBT inclusion, with new policies that protect and respect the rights of the LGBT community."
          Keywords: LGBT inclusion, rights, respect, protection, government policies.

      ESP15: Government Promoting Tolerance and Combatting Discrimination.
          Example: "Efforts to promote tolerance and fight discrimination are creating a more inclusive society in El Salvador."
          Keywords: tolerance, anti-discrimination, inclusivity, social progress, government efforts.

      ESP16: Strengthening Institutions and Creating Transparency Mechanisms to Combat Corruption.
          Example: "The government has strengthened its institutions and implemented transparency mechanisms to combat corruption, ensuring accountability at all levels."
          Keywords: strengthened institutions, transparency, anti-corruption, accountability, reforms.

      ESP17: Public Support for Anti-Corruption Initiatives Increasing Trust in Institutions.
          Example: "El Salvador's anti-corruption efforts are gaining widespread public support, restoring trust in governmental institutions."
          Keywords: anti-corruption, public support, institutional trust, transparency, government integrity.

    Instructions for Classification:

    1- Read carefully the input {input_text}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed.
    3- Generate a Json structure:
              (
                "classification": ["ESP1", "ESP2", "ESP3", "ESP4", "ESP5", "ESP6", "ESP7", "ESP8", "ESP9", "ESP10", "ESP11", "ESP12", "ESP13", "ESP14", "ESP15","ESP16", "ESP17"]
                 at most 2 categories,
                "reasoning": reasoning of the answer in maximum 50 words.
              ).
        """
        self.prompt_ES_agent = PromptTemplate(template=self.ES_agent, input_variables=["input_text"])

        self.llm_chain_ES_agent = LLMChain(prompt=self.prompt_ES_agent, llm=self.client)

    def _run_agent(self, input_text):
        try:
            ES_output = self.llm_chain_ES_agent.run({"input_text": input_text})
            return ES_output
        except Exception as e:
            print(e)
            return "Error ES Positive agent"


# 2. Negative Sentiment Narratives
class ES_Agent_Negative(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.ES_agent = """
    Your task is to analyze and categorize a social media post {input_text} from El salvador according to predefined narrative themes that reflect positive sentiment with validated context.


    Narratives to Detect:

      ESN1: The Government’s Rigid Approach is Compromising Civil Liberties.
      Example: "The government’s strict security policies are being criticized for compromising civil liberties under the guise of law and order."
      Keywords: civil liberties, law and order, government rigidity, criticism, human rights.

      ESN2: Security Policies Lean Towards Authoritarianism, Risking Escalation of Violence.
          Example: "El Salvador’s security measures are showing authoritarian tendencies, increasing concerns that violence could escalate instead of being reduced."
          Keywords: authoritarianism, security policies, violence, civil unrest, escalation.

      ESN3: Government Actions Attracting International Scrutiny, Deterring Investment.
          Example: "El Salvador is facing international scrutiny over its policies, and there are concerns that foreign investment may be deterred."
          Keywords: international scrutiny, government policies, deterred investment, foreign relations, economic risk.

      ESN4: Cryptocurrency Policies are Risky, Endangering Economic Stability.
          Example: "El Salvador’s decision to adopt Bitcoin is seen as risky, potentially threatening the country’s economic stability due to speculative investments."
          Keywords: cryptocurrency, Bitcoin, economic risk, speculative investments, financial instability.

      ESN5: Bitcoin Policy Isolating the Country from Traditional Financial Allies.
          Example: "The Bitcoin policy is creating uncertainty, as El Salvador becomes more isolated from its traditional financial partners."
          Keywords: Bitcoin policy, financial isolation, uncertainty, traditional finance, international business.

      ESN6: Push for Bitcoin Deepens Economic Vulnerability.
          Example: "The push for Bitcoin could deepen El Salvador's economic vulnerability by increasing dependence on volatile digital assets."
          Keywords: Bitcoin, economic vulnerability, dependence, volatility, financial instability.

      ESN7: El Salvador’s Distancing from International Partners is Alienating Allies.
          Example: "El Salvador’s foreign policies are alienating its regional allies, weakening its influence on the international stage."
          Keywords: foreign policy, international relations, alienation, regional allies, diminished influence.

      ESN8: Independent Policies Increase Isolation from Key Support Networks.
          Example: "By pursuing independent policies, El Salvador is detaching itself from vital support networks, increasing its international isolation."
          Keywords: independent policies, isolation, support networks, foreign relations, government strategy.

      ESN9: Alternative Media is Promoting Polarizing Narratives.
          Example: "Some alternative media outlets in El Salvador are fostering polarizing narratives, raising questions about the reliability of the information they provide."
          Keywords: alternative media, polarizing narrative, reliability, media trust, information control.

      ESN10: Government-Controlled Media Reduces Transparency.
          Example: "By controlling its own media channels, the government may be reducing transparency and limiting access to independent sources of information."
          Keywords: government control, media, reduced transparency, limited information, censorship concerns.

      ESN11: Government Reforms are Creating Economic Uncertainty.
          Example: "Recent government reforms have created uncertainty in the market, posing high risks that threaten the country’s long-term stability."
          Keywords: government reforms, economic uncertainty, high risk, market instability, future concerns.

      ESN12: Insufficient Gender Policies Allow Discrimination Against Women to Persist.
          Example: "Despite reforms, violence and discrimination against women continue, showing that the government's gender policies are insufficient."
          Keywords: gender policies, insufficient reform, violence against women, discrimination, government failure.

      ESN13: Lack of Funding Weakens Support Programs for Women’s Rights.
          Example: "Government programs aimed at protecting women’s rights are underfunded, severely limiting their effectiveness."
          Keywords: women’s rights, underfunding, lack of support, program effectiveness, gender equality.

      ESN14: Lack of Legal Protection for the LGBT Community Increases Vulnerability.
          Example: "Without strong legal protections, the LGBT community in El Salvador faces significant vulnerabilities, limiting their full participation in society."
          Keywords: LGBT rights, legal protection, vulnerability, exclusion, government policy.

      ESN15: Government Criticized for Failing to Actively Promote LGBT Rights.
          Example: "Current government policies have been criticized for not doing enough to actively promote the rights of the LGBT community, leading to an atmosphere of exclusion."
          Keywords: LGBT rights, government inaction, exclusion, criticism, equality.

      ESN16: Corruption Allegations Weaken Public Trust.
          Example: "Persistent corruption allegations against government officials are eroding public trust and undermining the effectiveness of institutions."
          Keywords: corruption, public trust, institutional effectiveness, government officials, allegations.

      ESN17: Lack of Transparency Raises Concerns About Government Integrity.
          Example: "The lack of transparency in government processes is raising concerns about the integrity and accountability of public management."
          Keywords: transparency, government integrity, accountability, concerns, public management.

    Instructions for Classification:

    1- Read carefully the input {input_text}.
    2- Determine which narrative(s) it supports based on the content and sentiment expressed.
    3- Generate a Json structure:
              (
                "classification": ["ESN1", "ESN2", "ESN3", "ESN4", "ESN5", "ESN6", "ESN7", "ESN8", "ESN9", "ESN10", "ESN11", "ESN12", "ESN13", "ESN14", "ESN15","ESN16","ESN17"]
                 at most 2 categories,
                "reasoning": reasoning of the answer in maximum 50 words.
              ).
        """
        self.prompt_ES_agent = PromptTemplate(template=self.ES_agent, input_variables=["input_text"])

        self.llm_chain_ES_agent = LLMChain(prompt=self.prompt_ES_agent, llm=self.client)

    def _run_agent(self, input_text):
        try:
            ES_output = self.llm_chain_ES_agent.run({"input_text": input_text})
            return ES_output
        except Exception as e:
            print(e)
            return "Error ES Negative agent"




# Intention dictionary to map intention codes to human-readable intentions
INTENTION_DICT = {
    # Positive Sentiment Narratives (ESP)
    "ESP1": "The Government’s Firm Approach Restoring Law and Order",
    "ESP2": "El Salvador is Becoming a Model of Security and Strong Leadership in Fighting Gang Violence",
    "ESP3": "Government Policies Attracting Support and International Investment",
    "ESP4": "El Salvador at the Forefront of Economic Modernization and Cryptocurrency Adoption",
    "ESP5": "Bitcoin Policy Opening Doors for Innovation and Attracting Entrepreneurs and International Investors",
    "ESP6": "The Bitcoin Revolution Could Break the Country’s Dependence on Traditional Financial Systems",
    "ESP7": "El Salvador Affirms Sovereignty and Emerges as a Regional Leader by Standing up to Foreign Pressure",
    "ESP8": "Independent Policies Lead to a New Era of Self-Sufficiency and Greater Regional Influence",
    "ESP9": "Alternative Media Providing Truth Suppressed by Traditional Media",
    "ESP10": "Government Promoting Transparency Through its Own Media Channels",
    "ESP11": "Economic Growth Promised by Government Reforms and Innovation",
    "ESP12": "Government Promoting Equality Policies and Empowering Women",
    "ESP13": "Support Programs for Vulnerable Women Reinforcing Commitment to Gender Equality",
    "ESP14": "Policies for LGBT Inclusion and Respect for Rights",
    "ESP15": "Government Promoting Tolerance and Combatting Discrimination",
    "ESP16": "Strengthening Institutions and Creating Transparency Mechanisms to Combat Corruption",
    "ESP17": "Public Support for Anti-Corruption Initiatives Increasing Trust in Institutions",

    # Negative Sentiment Narratives (ESN)
    "ESN1": "The Government’s Rigid Approach is Compromising Civil Liberties",
    "ESN2": "Security Policies Lean Towards Authoritarianism, Risking Escalation of Violence",
    "ESN3": "Government Actions Attracting International Scrutiny, Deterring Investment",
    "ESN4": "Cryptocurrency Policies are Risky, Endangering Economic Stability",
    "ESN5": "Bitcoin Policy Isolating the Country from Traditional Financial Allies",
    "ESN6": "Push for Bitcoin Deepens Economic Vulnerability",
    "ESN7": "El Salvador’s Distancing from International Partners is Alienating Allies",
    "ESN8": "Independent Policies Increase Isolation from Key Support Networks",
    "ESN9": "Alternative Media is Promoting Polarizing Narratives",
    "ESN10": "Government-Controlled Media Reduces Transparency",
    "ESN11": "Government Reforms are Creating Economic Uncertainty",
    "ESN12": "Insufficient Gender Policies Allow Discrimination Against Women to Persist",
    "ESN13": "Lack of Funding Weakens Support Programs for Women’s Rights",
    "ESN14": "Lack of Legal Protection for the LGBT Community Increases Vulnerability",
    "ESN15": "Government Criticized for Failing to Actively Promote LGBT Rights",
    "ESN16": "Corruption Allegations Weaken Public Trust",
    "ESN17": "Lack of Transparency Raises Concerns About Government Integrity"
}


# Build Agent Framework
class ClassAgent():
    def __init__(self, llm_35):
        self.llm_35 = llm_35
        self.class_agent = """ You are an agent with the task of analyzing a tweet input {input} from El Salvador.
        you are tasked with identifying entities mentioned or refered to in the input and generate and optimized query to launch a websearch
        concerning the input.

        optimal web query.

        Instructions:

            1- Read the input carefully.

            2- Indentify the real entities in it (actors or institutions from El salvador).

            3-Segment the original input depending on the entity they refer to.

            4- Reframe the tweet for an optimal websearch in Spanish.

            5- Generate a Json output following this pattern:

            (

            "entities": [Indentify the real entities in it (actors or institutions from El salvador)in the tweet as a list],
            "segments":[segments of the original input divided by the entity refered to]
            "search": Reframe the tweet for an optimal websearch in Spanish,
            )
            """
        self.prompt_class_agent = PromptTemplate(template=self.class_agent, input_variables=["input"])

        # Use LLMChain here with the knowledge it may be deprecated
        self.llm_chain_class_agent = LLMChain(prompt=self.prompt_class_agent, llm=self.llm_35)

    def _run_class_branch(self, input):
        try:
            class_agent_output = self.llm_chain_class_agent.run({"input": input})
            return class_agent_output
        except Exception as e:
            print(e)
            return "Error class layer"


class SentimentAgent():
    def __init__(self, llm_35):
        self.llm_35 = llm_35

        self.sentiment_agent = """
      You are an expert agent that has to analyze and classify a tweet {input} from El Salvador according to its sentiment.
      Your job is to categorize each tweet into one of three categories based on its sentiment: 1 (positive) or -1 (negative).



      Guidelines:
      . 1 (Positive): The tweet expresses a positive sentiment, such as happiness, praise, or admiration.
      .-1 (Negative): The tweet expresses a negative sentiment, such as criticism, disappointment, or disapproval.

        Instructions:

            - Read the input carefully.

            - Analize the context

            - Determine the sentiment expressed based on the content taking into consideration the context is El Salvador.

            - Generate a Json output following this pattern:

                    (
                      "label": 1,-1.
                      "reasoning: reasoning of the answer in 50 words.
                    ).
        """
        self.prompt_sentiment_agent = PromptTemplate(template=self.sentiment_agent, input_variables=["input"])

        # Use LLMChain
        self.llm_chain_sentiment_agent = LLMChain(prompt=self.prompt_sentiment_agent, llm=self.llm_35)

    def _run_sentiment_branch(self, input):
        try:
            sentiment_agent_output = self.llm_chain_sentiment_agent.run({'input': input})
            return sentiment_agent_output
        except Exception as e:
            print(e)
            return "Error sentiment layer"

class ContextBuilderAgent():
    def __init__(self, llm_35):
        self.llm_35 = llm_35

        self.context_builder_agent = """you are an agent with the task of creating a context from a social media input {input} from el Salvador.
        you are provided with the result of a web {search_output},

        your job is to :
        1- Review the input {input} and search output {search_output},
        2- Verify all the entries of the search_output {search_output} one by one and analize the aligment with the input {input},
        3- Build a context from the search_output to help understand the context of the input
        4-Provide a json output :
        (
        "context": Context to help understand the input.
         )

      just provide a json output
        """
        self.prompt_context_builder_agent = PromptTemplate(template=self.context_builder_agent, input_variables=["input","search_output"])

        # Corrected the LLMChain instantiation
        self.llm_chain_context_agent = LLMChain(prompt=self.prompt_context_builder_agent, llm=self.llm_35)

    def _run_context_builder_branch(self,input,search_output):
        self.input=input
        self.search_output=search_output

        try:
            context_agent_builder_output = self.llm_chain_context_agent.run({'input':self.input,'search_output':self.search_output})
            return context_agent_builder_output
        except Exception as e:
            print(e)
            return "Error context building layer"




# Class to define the overall processing pipeline
class VerificationPipeline:
    def __init__(self, llm_35, llm_4, input):
        self.class_agent = ClassAgent(llm_35)
        self.sentiment_agent = SentimentAgent(llm_35)
        #self.context_building_agent = ContextBuilderAgent(llm_35)
        self.llm_4 = llm_4
        self.llm_35 = llm_35
        self.input = input
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def process_news(self):
        self.logger.info('Process Initialized')

        # Entity extraction and search classification
        classification_result = self.classify_news(self.input)
        if not classification_result or 'error' in classification_result:
            self.logger.error(f"Classification failed: {classification_result}")
            return classification_result

        # Extract entities and segments
        entities = classification_result.get("entities", [])
        segments = classification_result.get("segments", [])
        search_input=classification_result.get("search", [])
        print("search",search_input)
        #search_output = info_extraction(search_input)
        # Context Building
        #context_build_result = self.build_context(self.input, search_output)
        #if 'error' in context_build_result:
            #return context_build_result

        #context = context_build_result.get("context", [])
        # Initialize final output container
        final_output = []

        # Loop through each entity and its corresponding segment
        for entity, segment in zip(entities, segments):
            self.logger.info(f"Processing entity: {entity}")

            # Sentiment analysis for the segment
            sentiment_analysis_result = self.analyze_sentiment(segment)
            if 'error' in sentiment_analysis_result:
                return sentiment_analysis_result
            print('Sentiment',sentiment_analysis_result)
            # Convert sentiment label (1 -> Positive, -1 -> Negative)
            sentiment_label = sentiment_analysis_result['label']
            sentiment_description = "Positive" if sentiment_label == 1 else "Negative"

            # Determine narrative intention (positive or negative)
            intention_result = self.determine_intention(sentiment_label, segment)
            if 'error' in intention_result:
                return intention_result
            print('Intent',intention_result )
            # Get narrative descriptions based on classification
            narrative_descriptions = [INTENTION_DICT.get(code, "Unknown Narrative") for code in intention_result['classification']]

            # Collect results for the current entity/segment
            entity_output = {
                "entity": entity,
                "segment": segment,
                "sentiment_label": sentiment_label,
                "sentiment_description": sentiment_description,
                "intention_classification": intention_result['classification'],
                "narrative_descriptions": narrative_descriptions,
                "reasoning": intention_result['reasoning'],
            }

            final_output.append(entity_output)

        # Return the aggregated results for all entities
        return final_output

    def classify_news(self, input):
        # Parsing output from class_agent
        result = self.class_agent._run_class_branch(input)
        print(f"ClassAgent output: {result}")  # Debugging: check raw output

        try:
            data = json.loads(result)  # Ensure the result is parsed as a dictionary
            return data
        except json.JSONDecodeError:
            self.logger.error(f"Error decoding classification result: {result}")
            return {"error": "Failed to classify the news."}

    def analyze_sentiment(self, input):
        # Parsing output from sentiment_agent
        result = self.sentiment_agent._run_sentiment_branch(input)
        try:
            data = json.loads(result)  # Ensure the result is parsed as a dictionary
            return data
        except json.JSONDecodeError:
            self.logger.error(f"Error decoding sentiment result: {result}")
            return {"error": "Failed to analyze sentiment."}

    def determine_intention(self, sentiment_label, input_text):
      # Select the appropriate agent based on sentiment
      if sentiment_label == 1:
          agent = ES_Agent_Positive(self.llm_35)
      elif sentiment_label == -1:
          agent = ES_Agent_Negative(self.llm_35)
      else:
          return {"error": "Unexpected sentiment label"}

      # Run the agent to get the intention
      result = agent._run_agent(input_text)

      # Check if the result is empty or not a valid string
      if not result or result.strip() == "":
          self.logger.error(f"Agent returned an empty result: {result}")
          return {"error": "Agent returned an empty result"}

      try:
          # If result is already a dict, no need to decode
          if isinstance(result, dict):
              return result

          # Check if it's a string and then decode it
          if isinstance(result, str):
              # Ensure it's a valid JSON string
              data = json.loads(result)
              return data

      except json.JSONDecodeError as e:
          # Log detailed error if the JSON is not properly formatted
          self.logger.error(f"Error decoding intention result: {result}, Error: {e}")
          return {"error": "Failed to determine the intention due to invalid JSON."}

      except Exception as e:
          # Catch any other unexpected error
          self.logger.error(f"Unexpected error: {e}")
          return {"error": "Failed to determine the intention due to an unexpected error."}




def extract_base_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path

    if domain.startswith("www."):
        domain = domain[4:]

    # Combine domain and the first part of the path for social media filtering
    if "facebook.com" in domain or "instagram.com" in domain or "twitter.com" in domain or "x.com" in domain:
        return domain + path.split('/')[1] + '/'

    return domain

def info_extraction(subject, length=500, api_key=serper_api_key, min_search=30):
    url = "https://google.serper.dev/search"


    payload = json.dumps({
        "q": subject,
        "gl": "sv",
        "hl": "es-419",
        "num": min_search,
        "tbs": 'qdr:w',
    })

    headers = {'X-API-KEY': api_key, 'Content-Type': 'application/json'}

    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        results = response.json()
        organic_results = results.get("organic", [])
    except requests.RequestException as e:
        print(f"Error during search: {e}")
        return [], [], []

    all_media_outlets, reliable_media, unreliable_media = [], [], []

    for result in organic_results:
        snippet = result.get('snippet')
        source_url = result.get('link')
        if snippet and source_url:
            base_domain = extract_base_domain(source_url)
            all_media_outlets.append(base_domain)

            if base_domain in RELIABLE_SOURCES:
                reliability_index = RELIABLE_SOURCES[base_domain]
                reliable_media.append({
                    "snippet": snippet[:length],
                    "source": source_url,  # Include the full link
                    "reliability_index": reliability_index
                })
            else:
                unreliable_media.append({
                    "snippet": snippet[:length],
                    "source": source_url  # Include the full link
                })
        else:
            print(f"Missing data in results: {result}")

    sorted_reliable = sorted(reliable_media, key=lambda x: x['reliability_index'], reverse=True)

    return all_media_outlets, sorted_reliable, unreliable_media

    def build_context(self, input, search_output):
        try:
            result = self.context_building_agent._run_context_builder_branch(input, search_output)
            self.logger.info(result)
            data = json.loads(result)
            context=data['context']
            return context
        except Exception as e:
            self.logger.error(f"Error during context building: {e}")
            return {"error": "Failed to build context."}


In [ ]:
# Example execution
input = "Hemos dado otro golpe contundente a la jerarquía de estos grupos terroristas mientras la oposicion se dedica a sabotear nuestro progreso."
verification_process = VerificationPipeline(llm_35, llm_4, input)
result = verification_process.process_news()
result

<ipython-input-7-5393687158e9>:290: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.llm_chain_class_agent = LLMChain(prompt=self.prompt_class_agent, llm=self.llm_35)
<ipython-input-7-5393687158e9>:294: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  class_agent_output = self.llm_chain_class_agent.run({"input": input})


ClassAgent output: 
{
"entities": ["grupos terroristas", "oposicion"],
"segments": [
"Hemos dado otro golpe contundente a la jerarquía de estos grupos terroristas mientras",
"la oposicion se dedica a sabotear nuestro progreso."
],
"search": "Golpe contundente a grupos terroristas El Salvador oposicion sabotear progreso"
}
search Golpe contundente a grupos terroristas El Salvador oposicion sabotear progreso
Sentiment {'label': 1, 'reasoning': 'The tweet expresses a positive sentiment as it highlights a successful strike against terrorist groups, showing progress and effectiveness in combating them. This can be seen as a positive development for the safety and security of the region.'}
Intent {'classification': ['ESP7', 'ESP16'], 'reasoning': "The post highlights El Salvador's success in combating terrorist groups, affirming sovereignty and resilience against foreign pressure (ESP7). It also implies a firm approach and commitment to law and order, potentially involving transparency mecha

[{'entity': 'grupos terroristas',
  'segment': 'Hemos dado otro golpe contundente a la jerarquía de estos grupos terroristas mientras',
  'sentiment_label': 1,
  'sentiment_description': 'Positive',
  'intention_classification': ['ESP7', 'ESP16'],
  'narrative_descriptions': ['El Salvador Affirms Sovereignty and Emerges as a Regional Leader by Standing up to Foreign Pressure',
   'Strengthening Institutions and Creating Transparency Mechanisms to Combat Corruption'],
  'reasoning': "The post highlights El Salvador's success in combating terrorist groups, affirming sovereignty and resilience against foreign pressure (ESP7). It also implies a firm approach and commitment to law and order, potentially involving transparency mechanisms to combat corruption (ESP16)."},
 {'entity': 'oposicion',
  'segment': 'la oposicion se dedica a sabotear nuestro progreso.',
  'sentiment_label': -1,
  'sentiment_description': 'Negative',
  'intention_classification': ['ESN1', 'ESN16'],
  'narrative_descri